In [1]:
import os 

os.chdir('C:/Users/David.Solano/OneDrive - Ipsos/Ipsos/Analisis de Audio/Piloto_180522')

## ASR 

Between a bunch of options I consider facebook model is the more accurate because of the volumen of data. However [here](https://huggingface.co/models?language=es&pipeline_tag=automatic-speech-recognition&sort=downloads) there are other alternatives to explore. 

In [2]:
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import re
import sys

model_name = 'jonatasgrosman/wav2vec2-large-xlsr-53-spanish'
#model_name = "facebook/wav2vec2-large-xlsr-53-spanish"
device = "cuda"

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'  # noqa: W605

model = Wav2Vec2ForCTC.from_pretrained(model_name)#.to(device)
processor = Wav2Vec2Processor.from_pretrained(model_name)

#ds = load_dataset("common_voice", "es", split="test[:1%]", data_dir="./cv-corpus-6.1-2020-12-11")

resampler = torchaudio.transforms.Resample(orig_freq=48_000, new_freq=16_000)

### For a list of paths

If we have a list of path the code will be as follows: 

In [ ]:
folder = './BASES/'

paths = os.listdir(folder)
audios = dict()
audios['path'] = [folder + path for path in paths]

In [8]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

df = pd.DataFrame(audios)
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())


hg_audio_dataset = Dataset(pa.Table.from_pandas(df))


def map_to_array(batch):
    speech, _ = torchaudio.load(batch["path"])
    batch["speech"] = resampler.forward(speech[0].squeeze(0)).numpy()
    batch["sampling_rate"] = resampler.new_freq
    return batch

hg_audio_dataset = hg_audio_dataset.map(map_to_array)

  0%|          | 0/5 [00:00<?, ?ex/s]

In [9]:
%%time
def map_to_pred(batch):
    features = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0], padding=True, return_tensors="pt")
    input_values = features.input_values#.to(device)
    attention_mask = features.attention_mask#.to(device)
    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits
    pred_ids = torch.argmax(logits, dim=-1)
    batch["predicted"] = processor.batch_decode(pred_ids)
    return batch

result = hg_audio_dataset.map(map_to_pred, batched=True, batch_size=16, remove_columns=list(hg_audio_dataset.features.keys()))

  0%|          | 0/1 [00:00<?, ?ba/s]

Wall time: 32min 22s


In [10]:
result['predicted']

['se lo daremos  señor y después la corona de españa',
 'llevas crema protectora',
 'bueno esto es unáurea prueba intentado descubrir que pasa con esto despech reconnition esperamos que todos halla bien os probar diferentes longitudes de audio por ejemplo esperamos que es dobdre treinta segundos y o saber que también hace la transcripción este modelito prueba número',
 'estoy raro amor ni siquierase que hablar aatayores dando el días carrrisstoy ramor no metiera do este aore sólo con que nobajo los ánimos apeopepuesto a escribir eso que te dije de de del texto en ingles  porque nunco se conocé como mejorarlo pero bueno al menos estoy practicando ya toela ahla charlalan la capacitación co angela y jayro eguros volivaroor se nos fueron como cuatro horas entonces angelamejo como mínimo pide ocho yo creque laba pedir unas doce horas contando como las cosas que yo antes usa para justificar como un poquito más de un día laboral ya porque sibujar stras cosas yo no sehjayrusili ha trabajado es